# Lib

In [6]:
%load_ext autoreload
%autoreload 2

import pandas as pd

import warnings

## ignore warning log in pandas
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 10000)


import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import joblib
import sys

if sys.platform=='win32':
    sys.path.insert(0,".\..\src")
        
elif sys.platform=='linux':
    sys.path.insert(0,"./../src")

from utils import utils_ml, parameters, metrics_evaluation, Pipeline_functions


from ipywidgets import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
if sys.platform=='win32':
    !dir .\
        
elif sys.platform=='linux':
    !ls ./

 O volume na unidade C � OS
 O N�mero de S�rie do Volume � C498-8BF5

 Pasta de c:\Users\caanp\OneDrive\Documents\repositories\ML-anti-fraud\notebooks

28/11/2022  00:07    <DIR>          .
23/11/2022  00:25    <DIR>          ..
10/09/2022  23:28    <DIR>          .ipynb_checkpoints
21/11/2022  23:51         1.007.073 1.EA.ipynb
25/11/2022  01:27            74.828 2.Feature_engineering.ipynb
25/11/2022  01:27            28.364 3.Stability_var.ipynb
27/11/2022  12:46           156.602 4.Modeling.ipynb
27/11/2022  12:46           156.602 5.Modelo_analisis_negocio.ipynb
25/11/2022  01:32           156.224 6.Pipeline.ipynb
25/11/2022  01:32           156.224 7.Pipeline_SPARK.ipynb
24/11/2022  23:44    <DIR>          catboost_info
               7 arquivo(s)      1.735.917 bytes
               4 pasta(s)   355.578.576.896 bytes dispon�veis


# Reading

In [3]:
## para saber en que directório uno se encuentra
#
if sys.platform=='win32':
    !echo %cd%
        
elif sys.platform=='linux':
    !pwd

c:\Users\caanp\OneDrive\Documents\repositories\ML-anti-fraud\notebooks


In [4]:
!dir .\\..\data

 O volume na unidade C � OS
 O N�mero de S�rie do Volume � C498-8BF5

 Pasta de c:\Users\caanp\OneDrive\Documents\repositories\ML-anti-fraud\data

24/11/2022  23:37    <DIR>          .
23/11/2022  00:25    <DIR>          ..
10/08/2021  12:08        26.677.890 dados.csv
28/11/2022  00:05        27.547.133 EA.pbix
22/11/2022  22:34            39.018 iv_woe_by_week.parquet
09/11/2022  10:35            54.091 MySQL_localhost_connection.pbix
24/11/2022  23:31         3.891.056 test_dataset.parquet
24/11/2022  23:31         8.711.592 train_dataset.parquet
25/11/2022  01:12        14.493.356 train_test_dataset_tranformed.parquet
24/11/2022  23:31         2.094.822 validation_dataset.parquet
               8 arquivo(s)     83.508.958 bytes
               2 pasta(s)   355.578.707.968 bytes dispon�veis


In [72]:
df_origin = pd.read_parquet(f"{parameters.data_path}\\dataset_structed.parquet")

In [29]:
scores = [x for x in range(0,101)]

## Cost Matrix and confusion Matrix

In [177]:
def cross_tab_matrices( df, label = "fraude", probability = "score", ticket = 'monto', cutoff = 0, total = False ):

    """
    
    
    """

    df_aval = df[ [label, probability, ticket] ]
    df_aval['predict'] = df_aval[probability].apply( lambda x: 1 if x>=cutoff else 0  )

    confusion_matrix = pd.crosstab( df_aval[label], df_aval['predict'], margins= total  )


    confusion_matrix_ticket = pd.crosstab( df_aval[label], df_aval['predict'], values = df[ticket], aggfunc="sum" , margins= total) 


    return confusion_matrix, confusion_matrix_ticket

In [199]:
def confusion_matrix_2_TP_FP_TN_FN(confusion_matrix, total =False):

    """


    """


    try:
        TP = confusion_matrix.iloc[lambda x: x.index == 1, lambda x: x.columns == 1][1][1]
    except:
        TP = 0

    try:
        FP = confusion_matrix.iloc[lambda x: x.index == 0, lambda x: x.columns == 1][1][0]
    except:
        FP = 0

    try:
        TN = confusion_matrix.iloc[lambda x: x.index == 0, lambda x: x.columns == 0][0][0]
    except:
        TN = 0

    try:
        FN = confusion_matrix.iloc[lambda x: x.index == 1, lambda x: x.columns == 0][0][1]
    except:
        FN = 0

    ## Computing the events and non events
    events = confusion_matrix.iloc[lambda x: x.index == 1].sum().sum()
    non_events = confusion_matrix.iloc[lambda x: x.index == 0].sum().sum()

    ## If total is True, we need to divide by 2
    P = events/2 if total == True else events
    N = non_events/2 if total == True else non_events


    return TP, FP, TN, FN, P, N

In [268]:
def confusion_matrix_2_bussines(confusion_matrix_ticket, total =False):

    """


    """

    try:
        gain = confusion_matrix_ticket.iloc[lambda x: x.index == 0, lambda x: x.columns == 0][0][0]
    except:
        gain = 0

    try:
        lost_by_bloq = confusion_matrix_ticket.iloc[lambda x: x.index == 0, lambda x: x.columns == 1][1][0]
    except:
        lost_by_bloq = 0

    try:
        lost_by_fraud = confusion_matrix_ticket.iloc[lambda x: x.index == 1, lambda x: x.columns == 0][0][1]
    except:
        lost_by_fraud = 0

    try:
        success_by_fraud = confusion_matrix_ticket.iloc[lambda x: x.index == 1, lambda x: x.columns == 1][1][1]
    except:
        success_by_fraud = 0


    total_ticket = confusion_matrix_ticket.sum().sum()
    
    ## We need to dividy by 4 when total is true.
    total_sell = total_ticket/4 if total==True else total_ticket


    return gain, lost_by_bloq, lost_by_fraud, success_by_fraud, total_sell



In [269]:
@interact

def slice_score_in_matrix_confusion(score = (0,100), option = [False, True]):


    confusion_matrix, confusion_matrix_monto = cross_tab_matrices(df_origin, cutoff= score, total = option)

    ######################### Métricas ##################################################
    TP, FP, TN, FN, P, N = confusion_matrix_2_TP_FP_TN_FN( confusion_matrix, total = option  )
    gain, lost_by_bloq, lost_by_fraud, success_by_fraud, total_sell = confusion_matrix_2_bussines(confusion_matrix_monto, total = option)

    Recall = TP/P
    TNR = TN/N

    indicator = (gain-lost_by_bloq-lost_by_fraud)/total_sell


    print("\n\n","========================== Confusion Matrix =======================================")
    display( confusion_matrix )


    print("Total fraudes: ", P)
    print("Total Não fraudes: ", N)
    print("\n\n")
    print("Recall: ", Recall)
    print("FPR: ", TNR)

    print("\n\n","========================== Cost confusion Matrix =======================================")
    display( confusion_matrix_monto  )

    print("Total vendido: ", total_sell)
    print("\n\n")
    print("indicador: ", indicator)


interactive(children=(IntSlider(value=50, description='score'), Dropdown(description='option', options=(False,…

# Requirements

In [110]:
path_root = parameters.root
!pip list --format=freeze > $path_root/requirements.txt

5697.85s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
